In [298]:
import torch
import torchtext
import torch.nn as nn
import pandas as pd
# from torchtext.datasets import text_classification
# NGRAMS = 2
# import os
# if not os.path.isdir('./.data'):
#     os.mkdir('./.data')
# train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](root='./.data', ngrams=NGRAMS, vocab=None)
# BATCH_SIZE = 16
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [287]:
def get_feature_from_one_xtrain(sample):
    result=[]
    summ=0
#     if len(sample.split())<200:
#         X=sample+" "
#         X=X*int(200/len(X.split()))
#         sample=X
    for word in sample.split():
#         for ch in word:
    
        summ+=1
        try: 
#             feature=np.sum(model.wv[word])
            result.append(tokenizer[word])
        except:
                feature=0
        if summ==50:
            return result
    return result

In [288]:
def get_feature_from_all_xtrain(samples):
    res=[]
    for sample in samples:
        s=get_feature_from_one_xtrain(sample)

#         if(len(s)<50):
            
#             for i in range(50-len(s)):
#                 x=np.zeros(300,dtype=np.float64)
#                 s.append(0)
            
# #         x=np.zeros(20,dtype=np.float64)
# #         for i in range(20):
# #             if(i <len(s)):
                
# #                 x[i]=s[i]
        res.append(s)
    return res

In [289]:
types=['__label__Sarcasm','__label__Positive','__label__Negative','__label__Neutral']

In [290]:
alldata=pd.read_csv('all_data_sentiment1.csv',names=['label','data'])
X_Train,Y_Train=data_train['data'],data_train['label']
all_sentences=alldata['data']

In [291]:
dictem={}
for i in range(len(types)):
    dictem[types[i]]=i
dictem

{'__label__Sarcasm': 0,
 '__label__Positive': 1,
 '__label__Negative': 2,
 '__label__Neutral': 3}

In [294]:
X_Train=get_feature_from_all_xtrain(all_sentences)
Y_Train=[dictem[i] for i in Y_Train]

In [295]:
allword=[]
for s in all_sentences:
    for word in s.split():
        allword.append(word)

f=set(allword)
len(f)
tokenizer=dict((word,index) for index,word in enumerate(f))

In [296]:
traindata=[(int(Y_Train[i]),torch.tensor(X_Train[i]).to(torch.int64))for i in range(len(Y_Train))]

In [297]:
train_dataset=traindata

In [299]:
print(train_dataset[:10])
print(device)


[(2, tensor([26835, 40281, 12522])), (2, tensor([35835, 37913,  7205, 16891, 34720, 36219, 30478, 30920, 46253, 31409,
        45949, 36223,  6849,  5418, 22416, 32964, 34013, 33152, 42863])), (2, tensor([32677, 12574, 10477, 42148, 37141, 30229, 45656,  6641, 32936, 23815,
        32549, 43542])), (2, tensor([ 3670, 23268,  7549, 36690])), (2, tensor([14806, 17167, 28542, 35611, 37504, 21593, 16817, 23147, 18175, 13160,
        23109, 23169,  4013, 14169, 33861,   500, 14085,   547, 23109,   510,
        29384, 16381, 45246, 42667, 23109, 36361, 18010, 23308,  5351, 17115,
        35650])), (2, tensor([25632, 37141, 22317, 39137, 24298, 27456,  9028, 11863, 14795, 48672,
        17268, 33983,  7029, 46226])), (2, tensor([ 1796, 39791,  8434, 31324, 33274,  4385, 11736, 36317, 21563, 26452,
         8695, 43112, 29266, 17252, 24679, 11994, 14255, 12830, 32685, 12763])), (2, tensor([37700, 30228, 45333,  4360, 45797, 26712,  4599])), (2, tensor([13868, 17192,  6975, 44093, 35049, 33612,

In [319]:

import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):

            
     
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [320]:
# train_dataset.get_labels()

In [321]:
VOCAB_SIZE =len(tokenizer)+1
EMBED_DIM = 80
NUN_CLASS = len(dictem.keys())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [322]:
model

TextSentiment(
  (embedding): EmbeddingBag(49375, 80, mode=mean)
  (fc): Linear(in_features=80, out_features=4, bias=True)
)

In [323]:

def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]

    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)

    return text, offsets, label

In [326]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=16, shuffle=True,collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):

        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=16, collate_fn=generate_batch)
    i=0
    for text, offsets, cls in data:
        i+=1
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
#             print(cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [328]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
train_len = int(len(train_dataset) * 0.80)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):
 
    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 2 seconds
	Loss: 0.0173(train)	|	Acc: 93.4%(train)
	Loss: 0.0002(valid)	|	Acc: 92.9%(valid)
Epoch: 2  | time in 0 minutes, 2 seconds
	Loss: 0.0122(train)	|	Acc: 95.6%(train)
	Loss: 0.0001(valid)	|	Acc: 92.5%(valid)
Epoch: 3  | time in 0 minutes, 2 seconds
	Loss: 0.0100(train)	|	Acc: 96.9%(train)
	Loss: 0.0001(valid)	|	Acc: 92.1%(valid)
Epoch: 4  | time in 0 minutes, 2 seconds
	Loss: 0.0085(train)	|	Acc: 97.7%(train)
	Loss: 0.0002(valid)	|	Acc: 90.6%(valid)
Epoch: 5  | time in 0 minutes, 2 seconds
	Loss: 0.0076(train)	|	Acc: 98.1%(train)
	Loss: 0.0002(valid)	|	Acc: 91.7%(valid)


In [336]:
def predict(sentence):
    output = model(torch.tensor(get_feature_from_all_xtrain([sentence])[0]), torch.tensor([0]))
    return types[output.argmax(1).item()]
predict(all_sentences[51])   

'__label__Negative'

In [317]:
len(list(sub_valid_))

5019

In [335]:
all_sentences[51]

'  لازم تصير مخالفه بكب زباله'

In [334]:
Y_Train[50]

2